# StyleAdv attack

Prerequisites: You need to add the pre-trained [style transfer models](https://drive.google.com/drive/folders/12ImHH2kJKw1Vs3rDUSRytP3DZYcHdsZw?usp=sharing) (by https://github.com/martiansideofthemoon/style-transfer-paraphrase) to your Google Drive.

Add a shortcut to your Google Drive under /My Drive/style_transfer_paraphrase

In [1]:
!git clone https://github.com/yifengd/StyleAttack.git
!pip install transformers >> /dev/null


Cloning into 'StyleAttack'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 376 (delta 13), reused 26 (delta 11), pack-reused 348
Receiving objects: 100% (376/376), 58.34 MiB | 18.18 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [80]:
!git -C /content/StyleAttack pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/yifengd/StyleAttack
   a2ede33..fd13a52  main       -> origin/main
Updating a2ede33..fd13a52
Fast-forward
 experiments/attack.py | 3 +++
 1 file changed, 3 insertions(+)


In [3]:
# Mount on Google Drive File System
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
%cd /content/StyleAttack/experiments

cmd = " ".join(["python",
              "attack.py",
              "--model_name", "textattack/distilbert-base-uncased-ag-news",
              "--bert_type", "distilbert-base-uncased",
              "--orig_file_path", "../data/clean/ag/test_100.tsv",
              "--model_dir", "/content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large",
              "--output_file_path", "/content/record_100.log"
])

!{cmd}


/content/StyleAttack/experiments
Some weights of the model checkpoint at /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.bias', 'transformer.extra_embedding_project.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0% 0/99 [00:00<?, ?it/s]/content/StyleAttack/experiments/style_paraphrase/inference_utils.py:95: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray wi

In [82]:

# !CUDA_VISIBLE_DEVICES=0 python attack.py --model_name textattack/distilbert-base-uncased-ag-news --bert_type distilbert-base-uncased --orig_file_path ../data/clean/ag/test_100.tsv --model_dir /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large --output_file_path /content/record_100.log


In [83]:
import pandas as pd

# dfs = []

# for label in label_set:
#   out_file = f'/content/record_100_{label}.log'
#   df = pd.read_table(out_file,sep='\t')
#   dfs.append(df)

# df = pd.concat(dfs, ignore_index=True)

# df

In [84]:
out_file = f'/content/record_100.log'
df = pd.read_table(out_file,sep='\t')

print(df.columns)

df = df.rename(columns={" orig_sent ": "original_text", " adv_sent ": "adversarial_text", 'p_val ': 'p_val', ' labels': 'labels'})
print(df.columns)

print(len(df))

df


Index(['p_val ', ' orig_sent ', ' adv_sent ', ' labels'], dtype='object')
Index(['p_val', 'original_text', 'adversarial_text', 'labels'], dtype='object')
87


,p_val,original_text,adversarial_text,labels
0,-1,unions representing workers at turner newal...,unions representing workers at turner newal...,2
1,-1,"space.com - toronto, canada -- a second\team ...","space.com - toronto, canada -- a second\team ...",3
2,-1,ap - a company founded by a chemistry researc...,ap - a company founded by a chemistry researc...,3
3,-1,ap - southern california's smog-fighting agen...,ap - southern california's smog-fighting agen...,3
4,-1,the british department for education and skil...,the british department for education and skil...,3
...,...,...,...,...
82,1,olympic champion kostas kederis today left ho...,today he was taken out of the hospital and se...,1
83,-1,"newcastle, england (ap) - england coach sven-...","newcastle, england (ap) - england coach sven-...",1
84,-1,"newcastle, england (ap) - striker emile heske...","newcastle, england (ap) - striker emile heske...",1
85,-1,new york (reuters) - staples inc. &lt;a href=...,new york (reuters) - staples inc. &lt;a href=...,2


In [85]:
# df.query("p_val == 1")

In [86]:
import os

out_name = "ag-news_100_styleadv_distilbert.csv"

out_dir = '/content/drive/MyDrive/AdversarialXAI/Adversarial Samples/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

df.to_csv(os.path.join(out_dir, out_name), index=False)